<a href="https://colab.research.google.com/github/erindonaher/ComputerSecurity/blob/main/Problem_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# Problem 6 code to decrypt a ciphertext using the DES algorithm with the provided key and prints the decrypted message.
def permute(input_string, table):
    # Rearrange the input string --> using table provided.
    result = ""
    for index in table:
        result += input_string[index - 1]
    return result

def xor_function(a, b):
    # bitwise xor operation between 2 binary strings.
    result = ""
    for bit_a, bit_b in zip(a, b):
        xor_bit = '0' if bit_a == bit_b else '1'
        result += xor_bit
    return result

def s_box_sub(xored, S_boxes):
    # Substitutes using s-boxes with dictionaries.
    s_box_result = ""
    for i in range(8):  # There are 8 S-boxes
        block = xored[i * 6:(i + 1) * 6]  # Each block is 6 bits
        row = int(block[0] + block[5], 2)  # First and last bits determine the row
        col = int(block[1:5], 2)  # Middle 4 bits determine the column
        val = S_boxes[i][row][col]  # Get the value from the S-box using the block directly
        s_box_result += format(val, '04b')  # 4-bit binary
    return s_box_result


def left_shift(k, shifts):
    # Shift the string k to the left by 'shifts' positions.
    shifted = k[shifts:] + k[:shifts]
    return shifted

def generate_round_keys(key, PC1, PC2, shifts):
    # round keys!
    round_keys = []
    key_permuted = permute(key, PC1)
    left = key_permuted[:28]
    right = key_permuted[28:]

    #for loop so I dont have to write out all the rounds!!
    for i in range(16):  # There are 16 rounds
        left = left_shift(left, shifts[i])
        right = left_shift(right, shifts[i])
        combined = left + right
        round_key = permute(combined, PC2)
        round_keys.append(round_key)
        #print("Round", i+1, "Key:", round_key)
    print("\n")
    return round_keys

def f_function(right, round_key, E, S, P):
    # Perform the f-function operations.
    expanded_right = permute(right, E)
    xored = xor_function(expanded_right, round_key)
    s_box_result = s_box_substitution(xored, S)
    result = permute(s_box_result, P)
    return result

def des_decrypt(ciphertext, round_keys, IP, IP_1, E, S, P):
    # Decrypt the ciphertext using the des algorithm.
    permuted_text = permute(ciphertext, IP)
    left = permuted_text[:32]
    right = permuted_text[32:]

    for i in range(15, -1, -1):  # Go through the rounds in reverse
        round_key = round_keys[i]
        print("Round", 16 - i, "\n\tKey:", round_key)

        right_expanded = f_function(right, round_key, E, S, P)
        new_right = xor_function(left, right_expanded)
        left = right  # Swap left and right for the next round
        right = new_right

        print("\tOutput of f function:", right_expanded)
        print("\tLn:", left)
        print("\tRn:", right)
    print("\n")
    combined = right + left  # Combine the final left and right parts
    decrypted = permute(combined, IP_1)  # Final permutation
    return binary_to_string(decrypted)

def binary_to_string(binary):
    # This function converts a binary string to ascii text.
    text = ""
    for i in range(0, len(binary), 8):  # Process 8 bits at a time
        byte = binary[i:i+8]
        text += chr(int(byte, 2))  # Convert binary to integer to character
    return text

# Definitions of ciphertext and key
key = "0100110001001111010101100100010101000011010100110100111001000100"
ciphertext = "1100101011101101101000100110010101011111101101110011100001110011"

# Generate round keys
round_keys = generate_round_keys(key, PC1, PC2, shifts)

# Decrypt the ciphertext
decrypted_message = des_decrypt(ciphertext, round_keys, IP, IP_1, E, S, P)

print("Decrypted Message:", decrypted_message)




Round 1 
	Key: 101000001001001000101010111100101000100010110001
	Output of f function: 01101010110011110011001101111111
	Ln: 00100111111011100101001110110101
	Rn: 11110001001111110000100111000101
Round 2 
	Key: 111000011001001000100010011011000110101001010101
	Output of f function: 00110100000100110111110010111111
	Ln: 11110001001111110000100111000101
	Rn: 00010011111111010010111100001010
Round 3 
	Key: 110000001000011000100011100111010110110010100010
	Output of f function: 11011011100001001010110000100000
	Ln: 00010011111111010010111100001010
	Rn: 00101010101110111010010111100101
Round 4 
	Key: 110000001010010100100100001110011001011100110110
	Output of f function: 10110110110000011111100010111100
	Ln: 00101010101110111010010111100101
	Rn: 10100101001111001101011110110110
Round 5 
	Key: 010000000110110000100100101100101111111001011000
	Output of f function: 01111000110001011101110101001111
	Ln: 10100101001111001101011110110110
	Rn: 01010010011111100111100010101010
Round 6 
	Key: 000